# AI for Autonomous Robots: Embed your algorithm on an edge device - Practical Session

## Introduction

In this practical session you will implement the conversion of a PyTorch MobileNetv3 to the ONNX format.




In [ ]:
# Install Dependencies
!pip install onnx
!pip install onnxruntime

## Import Libraries

In [ ]:
import torch
from torchvision import models
import cv2
import numpy as np
from torchsummary import summary
import onnxruntime as ort
import onnx
from matplotlib import pyplot as plt

## Create, print, and save a model

In [ ]:
# create model
model = models.mobilenet_v3_small(pretrained=True).cuda()
model.eval()

# print the model
summary(model, (3, 224, 224))

# save the model
torch.save(model, 'mobilenet_v3.pth')

## Prediction

In [ ]:
# load an image
img_path = '/content/img.jpg'
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()


# normalization
img = cv2.resize(img, (224, 224))
mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
img = np.array((img / 255.0 - mean) / std, dtype=np.float32)


img = img.transpose([2, 0, 1])
img_ = torch.from_numpy(img).unsqueeze(0).cuda()

# prediction
pred = model(img_)

# sort 
_, indices = torch.sort(pred, descending=True)
original_percentage = torch.nn.functional.softmax(pred, dim=1)[0] * 100
print(["prediction: {}, probability:{};".format(idx, original_percentage[idx].item()) for idx in indices[0][:3]])

## Task1: model conversion

In [ ]:
# Task1: convert the model to the ONNX format using torch.onnx

def to_onnx(model, f: str = None):
  # write your function here
  # official document https://pytorch.org/docs/stable/onnx.html#
  # -------
  
  input_names = [ "input1" ]
  output_names = [ "output1" ]

  if f is None:
    f = "mobilenet_v3.onnx"

  torch.onnx.export(model, img_, f, verbose=True, input_names=input_names, output_names=output_names)
  # -------
  
to_onnx(model)

## Task2: Infer the ONNX model

In [ ]:
# Task2: run the ONNX model using onnxruntime
def run_onnx(onnx_model, img):
  # write your function here
  # you can also get hints from the official document
  
  # -------
  ort_session = ort.InferenceSession(onnx_model)

  outputs = ort_session.run(
    None,
    {"input1": img},
  ) 
  # -------
  return outputs

def np_softmax(x):
    exp_x = np.exp(x)
    return exp_x/np.sum(exp_x)


result = run_onnx('mobilenet_v3.onnx', img_.cpu().numpy())
result = result[0]
indices = (-result).argsort()[:1000]
onnx_percentage = np_softmax(result) * 100
print(["onnx_percentage    : {}, probability:{};".format(idx, onnx_percentage[0][idx].item()) for idx in indices[0][:3]])
print(["original prediction: {}, probability:{};".format(idx, original_percentage[idx].item()) for idx in indices[0][:3]])

## Task3
Please include your answers in your reports.

In [ ]:
# Task3: Visualize the exported ONNX model using Netron https://netron.app/, and 
# 1. compare its architecture with the one we printed earlier.
# 2. compare the sizes of the models.
# Please write down your thoughts.